## Libraries

In [56]:
import os
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import  f_classif,VarianceThreshold,SelectKBest
from sklearn.linear_model import Perceptron
from sklearn.pipeline import Pipeline
import pickle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [6]:
os.chdir("C:/Users/u32118508/OneDrive - UPEC/Telechargement/pipeline_deploiement")
df1=pd.read_excel("data_segmentation.xlsx", sheet_name="segmentation")
df1.info()
df1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2010 entries, 0 to 2009
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   region_centroid_col   2010 non-null   int64  
 1   region_centroid_row   2010 non-null   int64  
 2   region_pixel_count    2010 non-null   int64  
 3   short_line_density_5  2010 non-null   float64
 4   short_line_density_2  2010 non-null   float64
 5   vedge_mean            2010 non-null   float64
 6   vegde_sd              2010 non-null   float64
 7   hedge_mean            2010 non-null   float64
 8   hedge_sd              2010 non-null   float64
 9   intensity_mean        2010 non-null   float64
 10  rawred_mean           2010 non-null   float64
 11  rawblue_mean          2010 non-null   float64
 12  rawgreen_mean         2010 non-null   float64
 13  exred_mean            2010 non-null   float64
 14  exblue_mean           2010 non-null   float64
 15  exgreen_mean         

,region_centroid_col,region_centroid_row,region_pixel_count,short_line_density_5,short_line_density_2,vedge_mean,vegde_sd,hedge_mean,hedge_sd,intensity_mean,rawred_mean,rawblue_mean,rawgreen_mean,exred_mean,exblue_mean,exgreen_mean,value_mean,saturation_mean,hue_mean,objet
0,77,186,9,0.0,0.0,2.00,2.67,2.33,3.56,34.96,31.11,43.44,30.33,-11.56,25.44,-13.89,43.44,0.30,-2.04,path
1,197,144,9,0.0,0.0,0.94,0.71,0.39,0.49,0.70,0.33,1.67,0.11,-1.11,2.89,-1.78,1.67,0.86,-1.99,window
2,74,129,9,0.0,0.0,0.22,0.03,0.11,0.03,0.52,0.00,1.56,0.00,-1.56,3.11,-1.56,1.56,1.00,-2.09,foliage
3,115,182,9,0.0,0.0,0.56,1.05,0.72,1.14,17.59,15.44,14.00,23.33,-6.44,-10.78,17.22,23.33,0.40,1.94,grass
4,187,120,9,0.0,0.0,2.22,2.08,1.28,0.65,60.04,56.89,69.33,53.89,-9.44,27.89,-18.44,69.33,0.22,-1.89,cement


##  Distribution de la variable d'interet

In [10]:
df1['objet'].value_counts()/sum(df1['objet'].value_counts())*100

foliage      14.527363
sky          14.527363
path         14.477612
cement       14.328358
brickface    14.328358
grass        14.029851
window       13.781095
Name: objet, dtype: float64

In [45]:
# split data
df1_train,df1_test=train_test_split(df1, test_size=300, random_state=1,stratify=df1['objet'])

# assess the split 
train=df1_train['objet'].value_counts()/sum(df1_train['objet'].value_counts())*100
test=df1_test['objet'].value_counts()/sum(df1_test['objet'].value_counts())*100
pd.merge(train.reset_index(),test.reset_index(), on='index').rename(columns={'objet_x': 'train_distri','objet_y': 'test_distri'})

,index,train_distri,test_distri
0,foliage,14.502924,14.666667
1,sky,14.502924,14.666667
2,path,14.502924,14.333333
3,brickface,14.327485,14.333333
4,cement,14.327485,14.333333
5,grass,14.035088,14.000000
6,window,13.801170,13.666667


## Création de Pipeline

In [60]:

modele=Pipeline([("selFirst", VarianceThreshold()),
                 ("selSecond", SelectKBest(score_func=f_classif,k=5)),
                 ("std",StandardScaler()),
                 ("clf", Perceptron(random_state=1))])

### Modélisation statsitique

In [72]:
modele.fit(df1_train.iloc[:,:-1], df1_train['objet'])

Pipeline(steps=[('selFirst', VarianceThreshold()),
                ('selSecond', SelectKBest(k=5)), ('std', StandardScaler()),
                ('clf', Perceptron(random_state=1))])

In [74]:
modele.named_steps["selFirst"].variances_

array([2.53000000e+02, 2.39000000e+02, 0.00000000e+00, 1.63880702e-03,
       5.25152662e-04, 7.62955103e+00, 9.91720000e+02, 1.39461841e+01,
       1.38633000e+03, 1.43440000e+02, 1.37110000e+02, 1.50890000e+02,
       1.42560000e+02, 5.68900000e+01, 9.41100000e+01, 5.61100000e+01,
       1.50890000e+02, 5.20085469e-02, 2.35249290e+00])

In [75]:
modele.named_steps["clf"].coef_

array([[ 47.42326748,   0.73059563, -42.32254741,  -7.79083535,
          1.81919089],
       [ 30.36267237,   1.2437223 , -27.937723  ,  -3.10041369,
         -2.32741686],
       [ -8.32744284,   1.69852151,   1.66215411,   2.46840213,
         -7.09102105],
       [ -0.82358334,  -1.50990521,   1.20286323,   0.66622233,
          4.07298331],
       [ 20.2100702 ,   2.87067494, -24.52914202,   2.00050795,
         -2.73593908],
       [  2.70401775,   1.95211258,   3.32829975,   2.10401509,
          2.20863406],
       [ -7.59726971,  12.48197747,  -5.44447293,  -6.98203754,
         -0.13675616]])

In [76]:
modele.named_steps["clf"].intercept_

array([ -3.,  -3.,  -7.,  -5.,  -4., -11.,  -3.])

In [90]:
predictions=modele.predict(df1_test.iloc[:,:-1])
accuracy_score(df1_test['objet'],predictions)

0.6066666666666667

In [99]:
parametres=[{'selSecond__k':[1,2,5,10, 15]}]

In [100]:
# grille de recherche en VC
# 5 validation, s' assurer que les données soient bien mélangées 
CvStrat=StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

grid=GridSearchCV(modele,param_grid=parametres,cv=CvStrat,refit=True)

In [101]:
grid.fit(df1_train.iloc[:,:-1], df1_train['objet'])

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             estimator=Pipeline(steps=[('selFirst', VarianceThreshold()),
                                       ('selSecond', SelectKBest(k=5)),
                                       ('std', StandardScaler()),
                                       ('clf', Perceptron(random_state=1))]),
             param_grid=[{'selSecond__k': [1, 2, 5, 10, 15]}])

In [102]:
pd.DataFrame(grid.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,{'selSecond__k': 1},0.297661
1,{'selSecond__k': 2},0.530409
2,{'selSecond__k': 5},0.560819
3,{'selSecond__k': 10},0.842690
4,{'selSecond__k': 15},0.886550


In [105]:
best_predictions=grid.best_estimator_.predict(df1_test.iloc[:,:-1])
accuracy_score(df1_test['objet'],best_predictions)

0.8866666666666667

### Exportation du modèles

In [110]:
f=open("pipeline_python.sav","wb")
pickle.dump(grid.best_estimator_,f)
f.close()